<a href="https://colab.research.google.com/github/Abhish2702/BTP-Depression-Detection-/blob/main/Copy_of_BTP_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199774 sha256=ccd30926cf77c730fc0e219b6a800ab7b1fde2d0cd86ca1375fb4e82e2c091ec
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [3]:
import fasttext
import fasttext.util

In [4]:

d_tweets_dataset=pd.read_csv("/content/clean_d_tweets.csv")
non_d_tweets_dataset=pd.read_csv("/content/clean_non_d_tweets.csv")
d_tweets_dataset.tweet = d_tweets_dataset.tweet.fillna('')
non_d_tweets_dataset.tweet = non_d_tweets_dataset.tweet.fillna('')
non_d_tweets_dataset.head()

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1424756177682927617,1424756177682927617,1.628523e+12,2021-08-09 05:35:18,-1000,NaN,touch pass be the new asmr,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1424741357592084485,1424741357592084485,1.628520e+12,2021-08-09 04:36:24,-1000,NaN,12 years ago i call my dad and he give me some...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1424739267553550337,1424739267553550337,1.628519e+12,2021-08-09 04:28:06,-1000,NaN,there be 6500 languages yet i choose to only s...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1424733524234366985,1424733524234366985,1.628518e+12,2021-08-09 04:05:17,-1000,NaN,hey everyone ask for a raise today i be tell y...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1424504791527411712,1424504791527411712,1.628463e+12,2021-08-08 12:56:22,-1000,NaN,i invest in 47 startups in a year here be how ...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [5]:
# d_tweets_dataset.head()
non_d_tweets_dataset.head()

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1424756177682927617,1424756177682927617,1.628523e+12,2021-08-09 05:35:18,-1000,NaN,touch pass be the new asmr,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1424741357592084485,1424741357592084485,1.628520e+12,2021-08-09 04:36:24,-1000,NaN,12 years ago i call my dad and he give me some...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1424739267553550337,1424739267553550337,1.628519e+12,2021-08-09 04:28:06,-1000,NaN,there be 6500 languages yet i choose to only s...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1424733524234366985,1424733524234366985,1.628518e+12,2021-08-09 04:05:17,-1000,NaN,hey everyone ask for a raise today i be tell y...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1424504791527411712,1424504791527411712,1.628463e+12,2021-08-08 12:56:22,-1000,NaN,i invest in 47 startups in a year here be how ...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [6]:
Depressed_Tweets=d_tweets_dataset['tweet']
Non_Depressed_Tweets=non_d_tweets_dataset['tweet']

In [7]:

Non_Depressed_Tweets.shape


(4687,)

In [8]:
def clean_text(text):
    # Remove punctuation marks
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text (split it into words)
    words = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.lower() not in stop_words]

    # Join the cleaned words back into a single string
    cleaned_text = ' '.join(words)

    return cleaned_text

In [ ]:
cleaned_data_depressed = [clean_text(text) for text in Depressed_Tweets]
cleaned_data_non_depressed=[clean_text(text) for text in Non_Depressed_Tweets]
# Print the cleaned data
for i, cleaned_text in enumerate(cleaned_data_depressed):
    print(f"Original Text {i + 1}: {Depressed_Tweets[i]}")
    print(f"Cleaned Text {i + 1}: {cleaned_text}")
    print()
for i, cleaned_text in enumerate(cleaned_data_non_depressed):
    print(f"Original Text {i + 1}: {Non_Depressed_Tweets[i]}")
    print(f"Cleaned Text {i + 1}: {cleaned_text}")
    print()


In [10]:
# cleaned_data.head()

In [11]:
cleaned_data_non_depressed

['touch pass new asmr',
 '12 years ago call dad give great advice short 1 min story',
 '6500 languages yet choose speak facts',
 'hey everyone ask raise today tell happen 1 say yes 2 say accelerate plan timeline 3 say learn stand fear awkward convo ask close mouth get feed',
 'invest 47 startups year math shake 4m deploy 47 company 85k avg check median valuation invest 16m let us think loud',
 'best people hire awesome pretty much immediately slow starters rarely become star',
 'get text make first million pod debut today want us talk',
 'paul tudor jones self make billionaire amp one greatest macro investors time say today bitcoin',
 'love cohort ltv great 2 reason ltv inform much spend customers view cohort tell get better worse time',
 'next ari gold sit college campus today public service announcement college hustler',
 'best startup idea week chrome extension pay',
 'trainer start clothe line perfect cut word wisdom weave come soon',
 'anyone want take course could afford sahil of

In [12]:
cleaned_data_depressed_np=np.array(cleaned_data_depressed)
cleaned_data_non_depressed_np=np.array(cleaned_data_non_depressed)

In [13]:
cleaned_data_non_depressed_np.shape

(4687,)

In [14]:
cleaned_data_depressed_reshaped=cleaned_data_depressed_np.reshape(3082,1)
cleaned_data_non_depressed_reshaped=cleaned_data_non_depressed_np.reshape(4687,1)
cleaned_data_total=np.append(cleaned_data_depressed_np,cleaned_data_non_depressed_reshaped)
cleaned_data_total

array(['real reason sad attach people distant pay attention people ignore make time people busy care people care less come let people go',
       'biggest problem overthinking everything',
       'worst sadness sadness teach hide', ...,
       'fan always ask watch old stuff finally answer legal',
       'ray romano hilarious comedian kind soul rare neurotic italian',
       'muellers report may finish mine next week johnny tremain due 1974'],
      dtype='<U261')

In [15]:
cleaned_data_total.shape
cleaned_data_total_reshaped=cleaned_data_total.reshape(7769,1)

In [16]:

cleaned_data_total_dataframe=pd.DataFrame(cleaned_data_total_reshaped)
cleaned_data_total_dataframe


,0
0,real reason sad attach people distant pay atte...
1,biggest problem overthinking everything
2,worst sadness sadness teach hide
3,make understand make anyone understand happen ...
4,think anyone really understand tire act okay a...
...,...
7764,cardi b want trademark catchphrase okurr think...
7765,bet kellyanne george conway pretty disturb mak...
7766,fan always ask watch old stuff finally answer ...
7767,ray romano hilarious comedian kind soul rare n...


In [17]:
y1=np.ones((3082,1),dtype=int)
y2=np.zeros((4687,1),dtype=int)
y3=np.append(y1,y2)

In [18]:
y3
Y=pd.DataFrame(y3)
Y

,0
0,1
1,1
2,1
3,1
4,1
...,...
7764,0
7765,0
7766,0
7767,0


In [19]:
X_train, X_test, y_train, y_test = train_test_split(cleaned_data_total_dataframe,Y ,
                                   random_state=104,
                                   test_size=0.25,
                                   shuffle=True)

In [20]:
X_train

,0
3587,apps desktopmobile
880,smile want cry talk want quiet pretend happy
2241,deep
7738,beyonce drop mueller report
2156,wonder one day one 40 second
...,...
654,time valuable waste ppl end worth never get back
5371,know organizations know expect expect minimum ...
729,keeper bad keep
5825,take moment acknowledge achievements big small...


In [21]:
y_train

,0
3587,0
880,1
2241,1
7738,0
2156,1
...,...
654,1
5371,0
729,1
5825,0


In [22]:
frames=[X_train,y_train]

In [23]:
result = pd.concat([X_train,y_train], axis=1, join='inner')
result.columns=["Text","Result"]

In [24]:
result.head()

,Text,Result
3587,apps desktopmobile,0
880,smile want cry talk want quiet pretend happy,1
2241,deep,1
7738,beyonce drop mueller report,0
2156,wonder one day one 40 second,1


In [25]:
result.to_csv("train.csv")

In [26]:
from google.colab import files

In [27]:
model1=fasttext.train_supervised(input="/content/train.csv")

In [28]:
result1=pd.concat([X_test,y_test],axis=1,join="inner")
result1.columns=["Text","Result"]


In [29]:
result1.head()

,Text,Result
5085,thank everyone read amp share tweet know world...,0
7478,pay box gym seem crazy meif want learn fight g...,0
2809,dark days since birth,1
2201,someone talk,1
3114,fun work vote make 100m year standup comedy ma...,0


In [30]:
result1.to_csv("test.csv")

In [31]:
files.download("test.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
model1.test("test.csv")

(0, nan, nan)

In [33]:
cleaned_data_total_dataframe.head()

,0
0,real reason sad attach people distant pay atte...
1,biggest problem overthinking everything
2,worst sadness sadness teach hide
3,make understand make anyone understand happen ...
4,think anyone really understand tire act okay a...


In [34]:
Y.head()

,0
0,1
1,1
2,1
3,1
4,1


In [35]:
dataset=pd.concat([cleaned_data_total_dataframe,Y],axis=1,join="inner")

In [36]:
dataset.head()

,0,0
0,real reason sad attach people distant pay atte...,1
1,biggest problem overthinking everything,1
2,worst sadness sadness teach hide,1
3,make understand make anyone understand happen ...,1
4,think anyone really understand tire act okay a...,1


In [37]:
dataset.columns=["text","result"]

In [38]:
dataset["text"].head()

0    real reason sad attach people distant pay atte...
1              biggest problem overthinking everything
2                     worst sadness sadness teach hide
3    make understand make anyone understand happen ...
4    think anyone really understand tire act okay a...
Name: text, dtype: object

In [39]:
dataset.head()

,text,result
0,real reason sad attach people distant pay atte...,1
1,biggest problem overthinking everything,1
2,worst sadness sadness teach hide,1
3,make understand make anyone understand happen ...,1
4,think anyone really understand tire act okay a...,1


In [41]:
texts=dataset['text'].tolist()

In [42]:
labels=dataset['result'].astype(str).tolist()

In [ ]:
texts

In [44]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [45]:
with open('train.txt', 'w', encoding='utf-8') as f:
    for text, label in zip(train_texts, train_labels):
        f.write(f'__label__{label} {text}\n')

with open('test.txt', 'w', encoding='utf-8') as f:
    for text, label in zip(test_texts, test_labels):
        f.write(f'__label__{label} {text}\n')


In [46]:
model = fasttext.train_supervised(input='train.txt', epoch=25, lr=1.0, wordNgrams=2, verbose=2)

In [49]:
# Evaluate the model on the test data
precision, recall, f1_score = model.test('test.txt')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')


Precision: 1554
Recall: 0.8558558558558559
F1 Score: 0.8558558558558559


In [50]:
with open('test.txt', 'r', encoding='utf-8') as f:
    test_data = f.readlines()

# Extract true labels from the test data
true_labels = [int(line.split()[0].replace('__label__', '')) for line in test_data]

# Predict labels using the trained model
predicted_labels = [int(model.predict(line.strip())[0][0].replace('__label__', '')) for line in test_data]

# Calculate accuracy
correct_predictions = sum(1 for true_label, predicted_label in zip(true_labels, predicted_labels) if true_label == predicted_label)
total_predictions = len(true_labels)
accuracy = correct_predictions / total_predictions

print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 85.59%
